# Genie 월간 Top100 크롤링

In [2]:
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm
import time, random
import requests

# 크롤링

## 크롤링할 링크 생성

In [4]:
# 날짜 리스트로 출력
start_date ='2022-01-01'
end_date = '2022-03-01'
dates = pd.date_range(start_date, end_date, freq='MS').to_list()
dates

[Timestamp('2022-01-01 00:00:00', freq='MS'),
 Timestamp('2022-02-01 00:00:00', freq='MS'),
 Timestamp('2022-03-01 00:00:00', freq='MS')]

In [5]:
# 크롤링할 날짜 생성
date_list = []
for one in dates:
    date = str(one).split(' ')[0].replace('-', '')
    date_list.append(date)

len(date_list)

3

In [6]:
date_list[:5]

['20220101', '20220201', '20220301']

In [7]:
# 링크 리스트
link_list = []
for one in date_list:
    temp1 = f'https://www.genie.co.kr/chart/top200?ditc=M&ymd={one}&hh=13&rtm=N&pg=1'
    temp2 = f'https://www.genie.co.kr/chart/top200?ditc=M&ymd={one}&hh=13&rtm=N&pg=2'
    link_list.append(temp1)
    link_list.append(temp2)

len(link_list)

6

In [8]:
link_list[:5]

['https://www.genie.co.kr/chart/top200?ditc=M&ymd=20220101&hh=13&rtm=N&pg=1',
 'https://www.genie.co.kr/chart/top200?ditc=M&ymd=20220101&hh=13&rtm=N&pg=2',
 'https://www.genie.co.kr/chart/top200?ditc=M&ymd=20220201&hh=13&rtm=N&pg=1',
 'https://www.genie.co.kr/chart/top200?ditc=M&ymd=20220201&hh=13&rtm=N&pg=2',
 'https://www.genie.co.kr/chart/top200?ditc=M&ymd=20220301&hh=13&rtm=N&pg=1']

In [9]:
len(link_list)/2

3.0

In [10]:
rank_list = []
for i in range(1, 101):
    rank_list.append(i)

In [11]:
# 페이지 접속
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}
# 지니뮤직 url
data = requests.get(link_list[0], headers=headers)
# soup : html형식
soup = bs(data.text, 'lxml')
soup.title.text

'음악, 그리고 설레임 - 지니'

In [12]:
link_list[0]

'https://www.genie.co.kr/chart/top200?ditc=M&ymd=20220101&hh=13&rtm=N&pg=1'

In [13]:
# 수집한 주간 제목
raw_title = soup.find('div', 'date')
raw_title

<div class="date">
<h3><span id="inc_date"></span></h3>
<a href="#" onclick="FG_calPopup.show($('#calendar_month')); return false;"><img alt="달력 보기" src="//image.genie.co.kr/imageg/web/btn/cal.png"/></a>
</div>

## 파일로 출력

In [14]:
num = int(len(link_list)/2)

for i in tqdm(range(num)):
    first = i*2
    second = i*2 + 1

    first_link = link_list[first]
    second_link = link_list[second]

    song_title_list = []
    artist_list = []
    album_list = []

    # 페이지 접속
    headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}
    # 지니뮤직 url
    data = requests.get(first_link, headers=headers)
    # soup : html형식
    soup = bs(data.text, 'lxml')

    # 곡 제목
    song_info = soup.find_all('td', 'info')
    for one in song_info:
        temp = one.find('a', 'title ellipsis').text.strip()
        song_title_list.append(temp)
    
    # 가수
    for one in song_info:
        temp = one.find('a', 'artist ellipsis').text
        artist_list.append(temp)

    # 앨범
    for one in song_info:
        temp = one.find('a', 'albumtitle ellipsis').text
        album_list.append(temp)

    # 페이지 접속
    headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}
    # 지니뮤직 url
    data = requests.get(second_link, headers=headers)
    # soup : html형식
    soup = bs(data.text, 'lxml')

    # 곡 제목
    song_info = soup.find_all('td', 'info')
    for one in song_info:
        temp = one.find('a', 'title ellipsis').text.strip()
        song_title_list.append(temp)
    
    # 가수
    for one in song_info:
        temp = one.find('a', 'artist ellipsis').text
        artist_list.append(temp)

    # 앨범
    for one in song_info:
        temp = one.find('a', 'albumtitle ellipsis').text
        album_list.append(temp)

    # 데이터 셋으로 만들기
    dict = {'순위':rank_list, '곡':song_title_list, '가수':artist_list, '앨범':album_list}
    df = pd.DataFrame(dict)

    # excel 파일로 출력
    date = date_list[i]
    df.to_excel(f'../crawling_data/month_genie/month_genie_{date}.xlsx', index=False)
    
    time.sleep( random.uniform(0.5, 0.9) )

100%|██████████| 3/3 [00:03<00:00,  1.13s/it]


# 코드 마지막